### Import required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from CartPole import CartPole
from CartPole_GPS import CartPole_GPS

from ilqr.dynamics import constrain
from copy import deepcopy

from EstimateDynamics import local_estimate
from GMM import Estimated_Dynamics_Prior

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Formulate the iLQR problem

In [2]:
'''
1 - dt = time step
2 - N = Number of control points in the trajectory
3 - x0 = Initial state
4 - x_goal = Final state
5 - Q = State cost
6 - R = Control cost
7 - Q_terminal = Cost at the final step
8 - x_dynamics array stores the information regarding system. 
    x_dynamics[0] = m = mass of the pendulum bob 
    x_dynamics[1] = M = mass of the cart 
    x_dynamics[2] = L = length of the massless rod 
    x_dynamics[3] = g = gravity 
    x_dynamics[4] = d = damping in the system
'''
dt = 0.005
N = 500  # Number of time steps in trajectory.
x_dynamics = np.array([1, 5, 2, 9.80665, 1]) # m=1, M=5, L=2, g=9.80665, d=1
x0 = np.array([-3.0, 0.0, 0.1, 0.0])  # Initial state
x_goal = np.array([2.0, 0.0, 0.0, 0.0])
# Instantenous state cost.
Q = np.eye(5)
Q[2, 2] = 10
Q[3, 3] = 10
# Q[4, 4] = 100
# Terminal state cost.
Q_terminal = 100 * np.eye(5)
# Instantaneous control cost.
R = np.array([[1.0]])

### iLQR on Cart Pole

In [3]:
cartpole_prob = CartPole(dt, N, x_dynamics, x0, x_goal, Q, R, Q_terminal)
xs, us = cartpole_prob.run_IterLinQuadReg()

iteration 0 accepted 21516.80057178476 [-1.95367333  1.78824251  2.75694394  3.97870727]
iteration 1 accepted 21489.037616539015 [-1.94995311  1.77800482  2.73567139  3.96586301]
iteration 2 failed 21489.037616539004 [-1.94995311  1.77800482  2.73567139  3.96586301]
iteration 3 failed 21489.037616539004 [-1.94995311  1.77800482  2.73567139  3.96586301]
iteration 4 failed 21489.037616539004 [-1.94995311  1.77800482  2.73567139  3.96586301]
iteration 5 failed 21489.037616539004 [-1.94995311  1.77800482  2.73567139  3.96586301]
iteration 6 accepted 21482.54024524796 [-1.94723925  1.77542215  2.72857893  3.96141387]
iteration 7 accepted 21481.967796399742 [-1.94653666  1.7743445   2.7261094   3.95985561]
iteration 8 failed 21481.967796399746 [-1.94653666  1.7743445   2.7261094   3.95985561]
iteration 9 accepted 21481.088851551867 [-1.94541974  1.77184738  2.72097584  3.95659997]
iteration 10 failed 21481.088851551875 [-1.94541974  1.77184738  2.72097584  3.95659997]
iteration 11 failed 214

In [4]:
# State matrix split into individual states. For plotting and analysing purposes.
t = np.arange(N + 1) * dt
x = xs[:, 0] # Position
x_dot = xs[:, 1] # Velocity
theta = np.unwrap(cartpole_prob.deaugment_state(xs)[:, 2])  # Theta, makes for smoother plots.
theta_dot = xs[:, 3] # Angular velocity

### Simulate the real system and generate the data
Cost matrices, initial position and goal position will remain same as the above problem. As it indicates one policy. But still the initial positions and goal positions must be passed explicitly to the function. But you don't need to pass cost matrices (assume penalty on the system is same), this is just used to use to calculate the cost of the trajectory. Correct control action must be passed. Parameter gamma indicates how much of original data you want to keep

Variance of the Gaussian noise will be taken as input from a Unif(0, var_range) uniform distribution. Inputs: x_initial, x_goal, u, n_rollouts, pattern='Normal', pattern_rand=False, var_range=10, gamma=0.2, percent=20

Pattern controls how the control sequence will be modified after applying white Guassian noise (zero mean).
- Normal: based on the correction/mixing parameter gamma generate control (gamma controls how much noise we want).
- MissingValue: based on the given percentage, set those many values to zero (it is implicitly it uses "Normal" generated control is used). 
- Shuffle: shuffles the entire "Normal" generated control sequence.
- TimeDelay: takes the "Normal" generated control and shifts it by 1 index i.e. one unit time delay.
- Extreme: sets gamma as zeros and generates control based on only noise.

If 'pattern_rand' is 'True' then we don't need to send the explicitly, it will chose one randomly for every rollout (default is 'False'). If you want to chose specific pattern then send it explicitly. 

In [5]:
x_rollout, u_rollout, local_policy, cost = cartpole_prob.gen_rollouts(x0, x_goal, us, n_rollouts=10, pattern_rand=True, var_range=10, gamma=0.2, percent=20)

### Local system dynamics/model estimate
loca_estimate: function takes the states (arranged in a special format, [x(t), u(t), x(t+1)]), no. of gaussian mixtures and no.of states.

In [6]:
model = Estimated_Dynamics_Prior(init_sequential=False, eigreg=False, warmstart=True, 
                 min_samples_per_cluster=20, max_clusters=50, max_samples=20, strength=1.0)
model.update_prior(x_rollout, u_rollout)
A, B, C = model.fit(x_rollout, u_rollout)

In [8]:
print(A.shape)
print(B.shape)
print(C.shape)

(501, 5, 6)
(501, 5)
(501, 5, 5)


### iLQR on estimated model
Here system dynamics is specified in a special way. We give the A, B, C matrices as input. These matrices comes from GMM and GPS theory. They are the mean/expected trajectory followed by the states which is represented by the mean & covariance (A, B, C) matrices of a Gaussian. Remaining all properties of the iLQR problem remains the same (cost, initial & goal state, time steps). 

In [ ]:
x_traj,u_traj = cartpole_prob.run_IterLinQuadReg_matrix(A, B, C)

### Plot

In [ ]:
# Control sequence
plt.plot(np.arange(us.shape[0]), us, 'r.', label='Original')
plt.plot(np.arange(us.shape[0]), u_rollout[0:N], 'b.', label='Corrupted')
plt.plot(np.arange(us.shape[0]), u_traj, 'g.', label='Estimated')
plt.xlabel('Time steps')
plt.ylabel('U')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(xs.shape[0]), xs[:, 2], 'r.', label='Original')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_rollout)[0:N+1, 2], 'b.', label='Corrupted')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_traj)[:, 2], 'g.', label='Estimated')
plt.xlabel('Time steps')
plt.ylabel('Theta')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(xs.shape[0]), xs[:, 0], 'r.', label='Original')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_rollout)[0:N+1, 0], 'b.', label='Corrupted')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_traj)[:, 0], 'g.', label='Estimated')
plt.xlabel('Time steps')
plt.ylabel('Pos')
plt.legend()
plt.show()

### GPS 

In [ ]:
dt = 0.005
N = 500  # Number of time steps in trajectory.
x_dynamics = np.array([1, 5, 2, 9.80665, 1]) # m=1, M=5, L=2, g=9.80665, d=1
# Instantenous state cost.
Q = np.eye(5)
Q[1,1] = 10
Q[2, 2] = 1
Q[3, 3] = 10
Q[4, 4] = 1
# Terminal state cost.
Q_terminal = 100 * np.eye(5)
# Instantaneous control cost.
R = np.array([[1.0]])

In [ ]:
x_train = []
u_train = []
for i in range(10):
    print('iteration is ',i)
    x0 = np.array([2, 0, 0.001*i , 0])  # Initial state
    x_goal = np.array([2, 0.0, 0.0, 0.0])
    cartpole_prob = CartPole(dt, N, x_dynamics, x0, x_goal, Q, R, Q_terminal)
    xs, us = cartpole_prob.run_IterLinQuadReg()
    t = np.arange(N + 1) * dt
    x = xs[:, 0] # Position
    x_dot = xs[:, 1] # Velocity
    theta = np.unwrap(cartpole_prob.deaugment_state(xs)[:, 2])  # Theta, makes for smoother plots.
    theta_dot = xs[:, 3] # Angular velocity
    x_rollout, u_rollout, local_policy, x_gmm, cost = cartpole_prob.gen_rollouts(x0, x_goal, us, 
                                     n_rollouts=20, pattern_rand=False, var_range=10, gamma=0.8, percent=20)
    model = local_estimate(x_gmm, components=5, NoOfstates=5)
    A, B, C = model.estimate(N=N)
    x_traj,u_traj = cartpole_prob.run_IterLinQuadReg_matrix(A, B, C)
    x_train.append(x_traj)
    u_train.append(u_traj)

In [ ]:
x_train1 = x_train[0][:-1]
u_train1 = u_train[0]
for i in range(1,9):
    x_train1 = np.vstack((x_train1,x_train[i][:-1]))
    u_train1 = np.vstack((u_train1,u_train[i]))

In [ ]:
u_gr = constrain(u_train1,-0.9,0.9)

In [ ]:
kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
        random_state=0).fit(x_train1, u_gr)

In [ ]:
gpr.score(x_train1,u_gr)

In [ ]:
u_pre = gpr.predict(xs)

In [ ]:
plt.plot(np.arange(us.shape[0]), constrain(us, -0.9, 0.9), 'r.', label='Original')
plt.plot(np.arange(us.shape[0]), constrain(u_pre[0:N],-0.9,0.9), 'b.', label='Global')
plt.plot(np.arange(us.shape[0]), constrain(u_traj, -0.9, 0.9), 'g.', label='Local')
plt.xlabel('Time steps')
plt.ylabel('U')
plt.legend()
plt.savefig('control.pdf')
plt.show()

In [ ]:
x_rollout00, u_rollout00, local_policy00, x_gmm00, cost00 = cartpole_prob.gen_rollouts(x0, x_goal, u_pre[:-1], n_rollouts=10, var_range=0, gamma=1, percent=0)

In [ ]:
plt.plot(np.arange(xs.shape[0]), xs[:, 0], 'r.', label='Original')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_rollout00)[0:N+1, 0], 'b.', label='Global')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_traj)[:, 0], 'g.', label='Local')
plt.xlabel('Time steps')
plt.ylabel('Pos')
plt.legend()
plt.savefig('position.pdf')
plt.show()

In [ ]:
plt.plot(np.arange(xs.shape[0]), xs[:, 2], 'r.', label='Original')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_rollout00)[0:N+1, 2], 'b.', label='Global')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_traj)[:, 2], 'g.', label='Local')
plt.xlabel('Time steps')
plt.ylabel('Theta')
plt.legend()
plt.savefig('theta.pdf')
plt.show()

In [ ]:
# plt.subplot(3,1,1)
# plt.plot(np.arange(us.shape[0]), constrain(us, -0.9, 0.9), 'r.', label='Original')
# plt.plot(np.arange(us.shape[0]), constrain(u_pre[0:N],-0.9,0.9), 'b.', label='GPS')
# plt.plot(np.arange(us.shape[0]), constrain(u_traj, -0.9, 0.9), 'g.', label='Estimated')
# plt.xlabel('Time steps')
# plt.ylabel('U')
# plt.legend()
# plt.title('Control action vs time')


plt.subplot(2,1,1)
plt.plot(np.arange(xs.shape[0]), xs[:, 0], 'r.', label='Original')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_rollout00)[0:N+1, 0], 'b.', label='Corrupted')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_traj)[:, 0], 'g.', label='Estimated')
plt.xlabel('Time steps')
plt.title('position vs time')
plt.ylabel('Pos')



plt.subplot(2,1,2)
plt.plot(np.arange(xs.shape[0]), xs[:, 2], 'r.', label='Original' , lw=2)
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_rollout00)[0:N+1, 2], 'b.', label='Corrupted' , lw=2)
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_traj)[:, 2], 'g.', label='Estimated', lw=2)
plt.xlabel('Time steps')
plt.title('theta vs time')
plt.ylabel('Theta')

plt.subplots_adjust(hspace=1.5)
plt.savefig('total.pdf')
plt.show()